In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import scanpy as sc
import os
import numpy as np
import pandas as pd
import squidpy as sq

import seaborn as sns
import anndata as ad
import matplotlib.colors as mcolors
# from preprocess_helpers import get_scVI_latent_representation, compute_local_mean, get_LR_pattern_representation, clean_genes_for_LR_GSE232025



/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata as ad
import scanpy as sc
import os
import numpy as np
import pandas as pd
import squidpy as sq
import os
from sklearn.preprocessing import LabelEncoder
import scprep
from scvi.model import SCVI
import torch
import torchsde
from torchdyn.core import NeuralODE
from tqdm import tqdm
from torchcfm.conditional_flow_matching import *
from torchcfm.models import MLP, GradModel
from torchcfm.utils import plot_trajectories, torch_wrapper
import pandas as pd
import seaborn as sns
from umap import UMAP
import torch.nn as nn
from torchdyn.core import NeuralODE
import seaborn as sns
from scvi.model import SCVI
import scvi
from scipy.stats import wasserstein_distance
from scipy.stats import energy_distance
from sklearn.metrics import r2_score
from sklearn.neighbors import NearestNeighbors
from scipy import sparse
import liana as li
import re


def compute_local_mean(scRNA, representation='X_pca', spatial_key='spatial', radius=50):
    """
    Compute mean vector of the chosen representation for spatial neighbors.
    
    Parameters:
        scRNA: AnnData object
        representation: str, key in .obsm, e.g. 'X_pca', 'X_umap', 'X_scVI'
        spatial_key: str, key in .obsm with spatial coordinates
        radius: float, neighborhood radius in same units as coordinates
    """
    coords = scRNA.obsm[spatial_key]
    X = scRNA.obsm[representation]

    nbrs = NearestNeighbors(radius=radius).fit(coords)
    neighbors_idx = nbrs.radius_neighbors(coords, return_distance=False)

    local_means = np.zeros_like(X)

    for i, idx in enumerate(neighbors_idx):
        if len(idx) > 0:
            local_means[i] = X[idx].mean(axis=0)
        else:
            local_means[i] = X[i]  

    # Store the result in .obsm
    scRNA.obsm[f"local_mean_{representation}"] = local_means

    return scRNA


def get_scVI_latent_representation(scRNA, cell_type_key, spatial_key):
        print ("Using scVI for input data")
        #read the data (again cause scVI requires unormalized data) and set up scVI
        le = LabelEncoder()
        scRNA.obs[cell_type_key] = le.fit_transform(scRNA.obs[cell_type_key])
        scvi.model.SCVI.setup_anndata(scRNA)
        model = SCVI(scRNA)
        model.train()
        latent = model.get_latent_representation()
        scRNA.obsm["X_scVI"] = latent
        scRNA = compute_local_mean(scRNA, representation='X_scVI', radius=50)
        return scRNA



def get_LR_pattern_representation(scRNA, nz_prop=0.1, resource_name='consensus'):

    li.ut.spatial_neighbors(scRNA, bandwidth=200, cutoff=0.1, kernel='gaussian', set_diag=True)
    lrdata_by_day= []
    for stage in scRNA.obs['day'].unique():
        adata_stage = scRNA[scRNA.obs['day'] == stage].copy()
        lrdata = li.mt.bivariate(adata_stage,
                    resource_name=resource_name, # NOTE: uses HUMAN gene symbols!
                    local_name='cosine', # Name of the function
                    global_name="morans", # Name global function
                    n_perms=100, # Number of permutations to calculate a p-value
                    mask_negatives=False, # Whether to mask LowLow/NegativeNegative interactions
                    add_categories=True, # Whether to add local categories to the results
                    nz_prop=nz_prop, # Minimum expr. proportion for ligands/receptors and their subunits
                    use_raw=False,
                    verbose=True
                    )
        # Save or analyze results for this stage
        # adata_stage.uns['liana_res'].to_csv(f'liana_results_stage_{stage}.csv')
        lrdata_by_day.append(lrdata)


    

    # Step 1: Collect all unique LR pairs across all time points
    all_lr_pairs = set()
    for lrdata in lrdata_by_day:
        all_lr_pairs.update(lrdata.var_names)

    # Convert to sorted list for consistent ordering
    all_lr_pairs = sorted(list(all_lr_pairs))
    print(f"Total unique LR pairs across all time points: {len(all_lr_pairs)}")

    # Step 2: Create LR_pattern feature vectors for each time point
    for i, stage in enumerate(scRNA.obs['day'].unique()):
        # Get the subset of cells for this stage
        stage_mask = scRNA.obs['day'] == stage
        n_cells_stage = stage_mask.sum()
        
        # Initialize LR_pattern matrix with zeros
        lr_pattern_matrix = np.zeros((n_cells_stage, len(all_lr_pairs)))
        
        # Get the lrdata for this stage
        lrdata = lrdata_by_day[i]
        
        # Convert sparse matrix to dense if needed
        if sparse.issparse(lrdata.X):
            lrdata_dense = lrdata.X.toarray()
        else:
            lrdata_dense = lrdata.X
        
        # Fill in the values for LR pairs that exist in this stage
        for j, lr_pair in enumerate(all_lr_pairs):
            if lr_pair in lrdata.var_names:
                # Get the index of this LR pair in the current lrdata
                lr_idx = lrdata.var_names.get_loc(lr_pair)
                # Copy the values for all cells
                lr_pattern_matrix[:, j] = lrdata_dense[:, lr_idx]
            # If LR pair doesn't exist in this stage, it remains 0 (already initialized)
        
        # Add LR_pattern to obsm for cells of this stage
        if 'LR_pattern' not in scRNA.obsm:
            # Initialize the full LR_pattern matrix for all cells
            scRNA.obsm['LR_pattern'] = np.zeros((scRNA.n_obs, len(all_lr_pairs)))
        
        # Fill in the values for this stage
        scRNA.obsm['LR_pattern'][stage_mask] = lr_pattern_matrix
        
        print(f"Stage {stage}: {n_cells_stage} cells, {len(lrdata.var_names)} LR pairs available")

    # Step 3: Add metadata about the LR pairs
    scRNA.uns['LR_pattern_pairs'] = all_lr_pairs
    print(f"LR_pattern matrix shape: {scRNA.obsm['LR_pattern'].shape}")
    print(f"LR pair names stored in scRNA.uns['LR_pattern_pairs']")

    return scRNA


def clean_genes_for_LR_GSE232025(scRNA):
    scRNA.var["gene_symbol_original"] = scRNA.var_names
    def clean_name(name):
        # keep leftmost alias before '|'
        primary = name.split("|")[0]
        # remove [hs], [nr], etc.
        primary = re.sub(r"\[.*?\]", "", primary)
        return primary.strip()
    scRNA.var_names = [clean_name(x) for x in scRNA.var_names]
    scRNA.var_names_make_unique()
    return scRNA

In [3]:
# import os, subprocess
# os.makedirs("MOSTA", exist_ok=True)
# # list of stages
# stages = ["E9.5", "E10.5", "E11.5", "E12.5", "E13.5", "E14.5", "E15.5", "E16.5"]
# # base URL pattern
# base_url = "https://ftp.cngb.org/pub/SciRAID/stomics/STDS0000058/stomics/{stage}_E1S1.MOSTA.h5ad"
# for stage in stages:
#     url = base_url.format(stage=stage)
#     out = f"MOSTA/{stage}.h5ad" 
#     print(f"Downloading {url} → {out}")
#     subprocess.run([
#         "wget", "--progress=bar:force", "-O", out, url
#     ])

In [4]:
def load_scRNA_GSE072025(data_path):
    """
    This function processes the GSE072025 dataset and returns the concatenated AnnData object.
    """
    sc1= sc.read_h5ad(os.path.join(data_path, "E15.h5ad"))
    sc1.obs["Batch"] = "0"
    sc2= sc.read_h5ad(os.path.join(data_path, "E16.h5ad"))
    sc2.obs["Batch"] = "1"
    sc3= sc.read_h5ad(os.path.join(data_path, "E17.h5ad"))
    sc3.obs["Batch"] = "2"
    sc4= sc.read_h5ad(os.path.join(data_path, "E18.h5ad"))
    sc4.obs["Batch"] = "3"
    sc5= sc.read_h5ad(os.path.join(data_path, "E19.h5ad"))
    sc5.obs["Batch"] = "4"

    adatas = [sc1, sc2, sc3, sc4, sc5]
    # adatas = [sc1, sc2, sc3, sc4]

    ## cell color mapping
    all_cell_types = set()
    for adata in adatas:
        all_cell_types.update(adata.obs["annotation"].unique())

    # Create a consistent color map for all cell types
    palette = sns.color_palette("tab20", len(all_cell_types))
    color_dict = dict(zip(sorted(all_cell_types), palette))

    for sci in adatas:
        sci.obs = sci.obs.rename(columns={"annotation": "celltype", "Batch": "day"})
        sci.obs["celltype"] = sci.obs["celltype"].astype(str)
        sci.obs["color"] = sci.obs["celltype"].map(color_dict)
        sci.obs["color"] = sci.obs["color"].apply(lambda x: mcolors.to_hex(x) if isinstance(x, (tuple, list, np.ndarray)) else x)
    scRNA = ad.concat(adatas, label='day', keys=[a.obs['day'][0] for a in adatas])

    return scRNA

In [5]:
data_path= "/Users/rssantanu/Desktop/codebase/constrained_FM/datasets/raw_datasets/GSE072025"

scRNA = load_scRNA_GSE072025(data_path)

/var/folders/mc/s2w117k11cj56552qbkf28zw0000gn/T/ipykernel_63035/2664813648.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


In [6]:
num_genes = 10000
n_comps = 50

In [7]:
sc.pp.highly_variable_genes(scRNA, batch_key='day', n_top_genes=num_genes, flavor='seurat_v3')
scRNA = scRNA[:, scRNA.var.highly_variable]
sc.tl.pca(scRNA, n_comps=n_comps)
sc.pp.neighbors(scRNA)
sc.tl.umap(scRNA)
# scRNA= compute_local_mean(scRNA, representation='X_pca')
# scRNA = get_scVI_latent_representation(scRNA, cell_type_key='celltype', spatial_key='spatial')

# scRNA = clean_genes_for_LR_GSE232025(scRNA)
# scRNA = get_LR_pattern_representation(scRNA, nz_prop=nz_prop)

/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:75: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/scanpy/preprocessing/_pca.py:314: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.


In [8]:
scRNA

AnnData object with n_obs × n_vars = 155684 × 10000
    obs: 'slice_ID', 'raw_x', 'raw_y', 'new_x', 'new_y', 'new_z', 'celltype', 'day', 'color'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'hvg', 'pca', 'neighbors', 'umap'
    obsm: 'X_umap', 'spatial', 'X_pca'
    varm: 'PCs'
    layers: 'raw_counts'
    obsp: 'distances', 'connectivities'

In [9]:
# scRNA= compute_local_mean(scRNA, representation='X_pca')
# scRNA = get_scVI_latent_representation(scRNA, cell_type_key='celltype', spatial_key='spatial')

# scRNA = clean_genes_for_LR_GSE232025(scRNA)
# scRNA = get_LR_pattern_representation(scRNA, nz_prop=nz_prop)

In [10]:
def clean_genes(scRNA):
    scRNA.var["gene_symbol_original"] = scRNA.var_names
    def clean_name(name):
        # keep leftmost alias before '|'
        primary = name.split("|")[0]
        # remove [hs], [nr], etc.
        primary = re.sub(r"\[.*?\]", "", primary)
        return primary.strip()
    scRNA.var_names = [clean_name(x) for x in scRNA.var_names]
    scRNA.var_names_make_unique()
    return scRNA

In [11]:
# Required columns: 'FBgn', 'gene_symbol' (fly), 'Human_gene' (HGNC symbol)
map_df = pd.read_csv('/Users/rssantanu/Desktop/codebase/constrained_FM/datasets/orthodb/hd_human_ortholog.csv')

# 1) Build a mapping from your var_names → human symbol
vn = pd.Index(scRNA.var_names)

if vn.str.startswith('FBgn').all():             # case: var_names are FBgn IDs
    key = 'FBgn'
elif vn.isin(map_df['gene_symbol']).mean() > 0: # case: var_names are fly symbols
    key = 'gene_symbol'
else:
    raise ValueError('scRNA.var_names must be FBgn or fly gene symbols present in the mapping CSV')

m = (map_df
     .dropna(subset=[key, 'Human_gene'])
     .drop_duplicates(subset=[key])
     .set_index(key)['Human_gene']
     .to_dict())

# 2) Keep only genes we can map, attach human symbols
keep_mask = vn.map(m).notna()
scRNA_h = scRNA[:, keep_mask].copy()
scRNA_h.var['human_symbol'] = scRNA_h.var_names.map(m)

# # 3) Collapse duplicates that map to the same human symbol (sum; use mean if preferred)
# X = scRNA_h.to_df()
# X.columns = scRNA_h.var['human_symbol'].values
# X = X.groupby(level=0, axis=1).sum()

# # 4) Rebuild AnnData with human symbols as var_names
# scRNA_h = sc.AnnData(X.values, obs=scRNA_h.obs.copy(),
#                      var=pd.DataFrame(index=X.columns))
# scRNA_h.var_names_make_unique()


In [12]:
scRNA_h.var.index= scRNA_h.var['human_symbol']
scRNA_h.var['gene_symbol_original'] = scRNA_h.var.index

scRNA_h.var_names = scRNA_h.var['human_symbol'].str.upper().values
# scRNA_h.var_names_make_unique()

In [13]:
# scRNA= clean_genes(scRNA)
nz_prop = 0.04
scRNA = get_LR_pattern_representation(scRNA_h, nz_prop=nz_prop, resource_name='Consensus')

/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Using `.X`!
Converting to sparse csr matrix!
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:381: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
6 samples of mat are empty, they will be removed.
Using resource `Consensus`.
100%|██████████| 100/100 [00:01<00:00, 93.10it/s]
/Users/rss

Total unique LR pairs across all time points: 25
Stage 0: 15295 cells, 9 LR pairs available
Stage 1: 14634 cells, 1 LR pairs available
Stage 2: 17787 cells, 2 LR pairs available
Stage 3: 64658 cells, 2 LR pairs available
Stage 4: 43310 cells, 22 LR pairs available
LR_pattern matrix shape: (155684, 25)
LR pair names stored in scRNA.uns['LR_pattern_pairs']


In [14]:
scRNA.var

,highly_variable,highly_variable_rank,means,variances,variances_norm,highly_variable_nbatches,human_symbol,gene_symbol_original
DRG2,True,5863.0,0.045535,0.047114,0.995427,5,DRG2,DRG2
YWHAH,True,6988.0,0.459065,0.457645,1.004768,5,YWHAH,YWHAH
YWHAH,True,4575.0,0.527510,0.455456,0.995208,4,YWHAH,YWHAH
TIMMDC1,True,4976.5,0.011289,0.011061,0.983560,4,TIMMDC1,TIMMDC1
TLR8,True,4233.0,0.012535,0.013021,1.015447,5,TLR8,TLR8
...,...,...,...,...,...,...,...,...
ZEB2,True,3987.0,0.018863,0.019786,1.030923,5,ZEB2,ZEB2
ZFHX4,True,4064.0,0.018945,0.019107,1.010127,5,ZFHX4,ZFHX4
MYH10,True,7661.5,0.148443,0.155619,0.951379,4,MYH10,MYH10
SPEG,True,4499.0,0.069261,0.072924,1.022816,5,SPEG,SPEG
